<a href="https://colab.research.google.com/github/subhashreegre/Web-Scraping-/blob/main/webscrap99acrestrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install ipywidgets beautifulsoup4 requests --quiet

In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files

In [9]:
def scrape_99acres(city_name, pages):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    filename = f"{city_name}_listings.csv"
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["Name", "Location", "Price"])
        writer.writeheader()

        for page in range(1, pages + 1):
            print(f"Scraping page {page} for {city_name.title()}...")
            url = f"https://www.99acres.com/search/property/buy/{city_name}-all?city=575&keyword={city_name}&page={page}"

            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"Failed to fetch page {page} for {city_name}")
                continue

            soup = BeautifulSoup(res.text, 'html.parser')
            listings = soup.find_all('div', class_='srpTuple__tupleTable')

            if not listings:
                print(f"No listings found on page {page} for {city_name}")
                break

            for listing in listings:
                try:
                    name = listing.find('a', class_='srpTuple__propertyName').text.strip()
                    location = listing.find('div', class_='srpTuple__propertyName').find_next_sibling().text.strip()
                    price = listing.find('td', class_='srpTuple__midGrid price').text.strip()

                    writer.writerow({
                        "Name": name,
                        "Location": location,
                        "Price": price
                    })
                except:
                    continue

            time.sleep(1.5)

    print(f"Done scraping {city_name.title()}. Downloading CSV...")
    files.download(filename)

In [10]:
# City selection
city_dropdown = widgets.SelectMultiple(
    options=["hyderabad", "bangalore", "mumbai", "delhi", "chennai"],
    value=["hyderabad"],
    description='Cities:',
    style={'description_width': 'initial'}
)

# Page count
page_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=10,
    step=1,
    description='Pages per city:',
    style={'description_width': 'initial'}
)

# Scrape button
scrape_button = widgets.Button(
    description="Start Scraping",
    button_style='success'
)

In [15]:
def on_scrape_button_click(b):
    clear_output()
    display(city_dropdown, page_slider, scrape_button)
    selected_cities = list(city_dropdown.value)
    pages = page_slider.value

    for city in selected_cities:
        scrape_99acres(city, pages)

scrape_button.on_click(on_scrape_button_click)
display(city_dropdown, page_slider, scrape_button)

SelectMultiple(description='Cities:', index=(2,), options=('hyderabad', 'bangalore', 'mumbai', 'delhi', 'chenn…

IntSlider(value=2, description='Pages per city:', max=10, min=1, style=SliderStyle(description_width='initial'…

Button(button_style='success', description='Start Scraping', style=ButtonStyle())

In [13]:
def scrape_99acres(city_name, pages):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    filename = f"{city_name}_listings.csv"
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["Name", "Location", "Price"])
        writer.writeheader()

        for page in range(1, pages + 1):
            print(f"\nScraping page {page} for {city_name.title()}...")
            url = f"https://www.99acres.com/search/property/buy/{city_name}-all?city=575&keyword={city_name}&page={page}"

            try:
                res = requests.get(url, headers=headers)
                if res.status_code != 200:
                    print(f"Failed to fetch page {page} for {city_name}, status code: {res.status_code}")
                    continue

                soup = BeautifulSoup(res.text, 'html.parser')

                # DEBUG: Check if listings are present
                listings = soup.find_all('div', {'class': lambda x: x and 'projectTuple' in x})
                print(f"Found {len(listings)} listings.")

                if not listings:
                    print("No listings found. Structure might have changed.")
                    continue

                for listing in listings:
                    try:
                        name = listing.find('a').text.strip()
                    except:
                        name = "N/A"
                    try:
                        location = listing.find('h2').text.strip()
                    except:
                        location = "N/A"
                    try:
                        price = listing.find('div', string=lambda t: t and '₹' in t).text.strip()
                    except:
                        price = "N/A"

                    writer.writerow({
                        "Name": name,
                        "Location": location,
                        "Price": price
                    })

                time.sleep(1.5)

            except Exception as e:
                print(f"Error scraping page {page}: {e}")
                continue

    print(f"\nDone scraping {city_name.title()}! File saved as: {filename}")
    files.download(filename)